In [5]:
import numpy as np

## Deriving P with method 4.1 from the paper

Consider a coordinate system centered in the center of the table, with $x$ and $y$ directions parallel to its edges. Once we have $K$, $R$ can be computed by considering that the rays through the projections of the vanishing points must be perpendicular.

Let $\mathbf u$ and $\mathbf v$ be the projections vanishing points on the plane $z=k$ in Euclidean coordinates (these can be obtained by inverting $K$ on the pixel coordinates). We know $\mathbf u \cdot \mathbf v = 0$, so solving for $k$ we get $k^2 = -x_u x_v - y_u y_v$. Immediately we get $\mathbf x' = \frac{\mathbf u}{||\mathbf u||}$ and $\mathbf y' = \frac {\mathbf v} {||\mathbf v||}$, and of course $\mathbf z' = \mathbf x' \times \mathbf y'$. Lastly,

$$
R = \begin{pmatrix} 
\\
\mathbf x' & \mathbf y' & \mathbf z' \\
\\
\end{pmatrix}
$$.

Once the intrinsics $\alpha$, $u_0$, $v_0$ and $f$ are known, the unit vector $\hat{\mathbf q}_a$ corresponding to the direction of the ray projected on a generic point $A$ on the picture can be computed. Let $u$ and $v$ be the (normalized) pixel coordinates of $A$ on the picture. By equation (2) on the paper, its projection on the image plane will be the vector $\mathbf q_a = \begin{pmatrix}\alpha_u (u-u_0) & k & \alpha_v(v_0 - v)\end{pmatrix}$. $\alpha_u$, $\alpha_v$ and $k$ are not known, but by dividing the vector by $\alpha_v$ a new vector on the same ray is obtained, whose coordinates are:

$$
\frac{\mathbf q_a}{\alpha_v} =
\begin{pmatrix}\frac{\alpha_u}{\alpha_v}(u-u_0) & \frac{k}{\alpha_v} & (v_0 - v)\end{pmatrix} =
\begin{pmatrix}\frac{1}{\alpha}(u-u_0) & f & (v_0 - v)\end{pmatrix}
$$

The unit vector $\hat{\mathbf q}_a$ corresponding to the ray can be obtained simply by normalizing the latter.

In [6]:
# All arguments are 3D vectors
def corner_lambda_h_coeff(qm_norm, qn_norm, qh_norm, qa_norm):
    return np.dot(np.cross(qn_norm, qm_norm), qh_norm)/np.dot(np.cross(qn_norm, qm_norm), qa_norm) * qa_norm

# Normalized vector in camera coordinates corresponding to point in pixel coordinates
def qnorm(u_px, v_px, u_0, v_0, alpha, f):
    q = np.array([(u_px - u_0)/alpha, v_px - v_0, f])
    return q/np.linalg.norm(q)

$\lambda_h$ can be derived imposing the area of the table surface as a constraint:

$$
AB \cdot AC = A \\
A = |\mathbf p_B - \mathbf p_A|\cdot|\mathbf p_C - \mathbf p_A| \\
A = |\lambda_h \mathbf c_B - \lambda_h \mathbf c_A|\cdot|\lambda_h \mathbf c_C - \lambda_h \mathbf c_A| \\
\lambda_h = \sqrt \frac{A}{|\mathbf c_B - \mathbf c_A|\cdot|\mathbf c_C - \mathbf c_A|}
$$

$\mathbf c_A$ is the vector obtained in equation (15) of the paper by decomposing $\mathbf t_{wc}$ as in equation (13):

$$
\mathbf p_A =
\frac{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \mathbf t_{wc}}{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \hat{\mathbf q}_a} \hat{\mathbf q}_a =
\frac{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \lambda_h \hat{\mathbf q}_h}{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \hat{\mathbf q}_a} \hat{\mathbf q}_a =
\lambda_h \frac{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \hat{\mathbf q}_h}{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \hat{\mathbf q}_a} \hat{\mathbf q}_a
=: \lambda_h \mathbf c_A
$$

In [7]:
def hom2eucl(v):
        assert(v[-1] != 0), f"Point at infinity {v} does not have an Euclidean correspondent!"
        return v[:-1]/v[-1]

In [8]:
# Vertex A of the table must be consecutive to both vertices B and C;
# H is the center of the table.
# vpx, vpy, uv* must be given in Euclidean pixel coordinates.
def find_projection_matrix2(uv_princ, alpha, f, vpx, vpy, uva, uvb, uvc, uvh, table_area):
    def normalize(v):
        return v/np.linalg.norm(v)
    qm_norm = qnorm(*vpx, *uv_princ, alpha, f)
    qn_norm = qnorm(*vpy, *uv_princ, alpha, f)
    qz_norm = np.cross(qm_norm, qn_norm)
    R = np.column_stack((qm_norm, qn_norm, qz_norm))
    
    qh_norm = qnorm(*uvh, *uv_princ, alpha, f)
    qa_norm = qnorm(*uva, *uv_princ, alpha, f)
    qb_norm = qnorm(*uvb, *uv_princ, alpha, f)
    qc_norm = qnorm(*uvc, *uv_princ, alpha, f)
    
    ca = corner_lambda_h_coeff(qm_norm, qn_norm, qh_norm, qa_norm)
    cb = corner_lambda_h_coeff(qm_norm, qn_norm, qh_norm, qb_norm)
    cc = corner_lambda_h_coeff(qm_norm, qn_norm, qh_norm, qc_norm)
    
    lambda_h = np.sqrt(table_area/(np.linalg.norm(cb - ca)*np.linalg.norm(cc - ca)))
    t = lambda_h * qh_norm
    
    Rt = np.column_stack((R, t))
    return Rt, ca*lambda_h, cb*lambda_h, cc*lambda_h

In [123]:
def norm_coordinates(v, normf):
    res = np.copy(v) 
    res[0:2] = res[0:2]/normf
    return res

uv_princ, f = (np.array([1.91780241, 2.60489289]), 1.9487965719680382)
alpha = 1

with open("rendered_markers/detected/outs.csv") as inf:
    inrows_name = [l.split(",") for l in inf.read().splitlines()]
    picnames = [row[-1] for row in inrows_name]
    inrows = np.array([row[:-1] for row in inrows_name]).astype("float")
vpxs, vpys, As, Bs, Cs, centers = inrows.reshape((12, -1, 3)).swapaxes(0,1)
normf = 1000
picname = "IMG_20190604_210132054.jpg"
img_idx = picnames.index(picname)
vpx = hom2eucl(norm_coordinates(vpxs[img_idx], normf))
vpy = hom2eucl(norm_coordinates(vpys[img_idx], normf))
uva = hom2eucl(norm_coordinates(As[img_idx], normf))
uvb = hom2eucl(norm_coordinates(Bs[img_idx], normf))
uvc = hom2eucl(norm_coordinates(Cs[img_idx], normf))
uvh = hom2eucl(norm_coordinates(centers[img_idx], normf))
table_area = 0.06237 # square meters
Rt, pa, pb, pc = find_projection_matrix2(uv_princ, alpha, f, vpx, vpy, uva, uvb, uvc, uvh, table_area)

In [124]:
def angle_between(v1, v2):
    return np.rad2deg(np.arccos(np.dot(v1, v2)/np.linalg.norm(v1)*np.linalg.norm(v2)))

db, dc = pa-pb, pa-pc
np.linalg.norm(db), np.linalg.norm(dc), angle_between(db, dc), angle_between(np.transpose(Rt[:,1]), np.transpose(Rt[:,0]))

(0.20338457537888416, 0.30666042340630423, 90.0435978540659, 90.46361165842058)

In [125]:
table_plane = np.append(Rt[:,2], -np.dot(Rt[:,2], Rt[:,3]))
table_plane

array([ 0.00091397,  0.4570229 ,  0.88941765, -0.11783063])

In [126]:
width_px = 4032
normf = 1000
fov_angle = np.rad2deg(2*np.arctan(width_px/(2*f*normf)))
fov_angle

91.94214459035763

In [127]:
uva

array([3.65427842, 2.66348481])

In [128]:
Rtc = np.row_stack((Rt, [0, 0, 0, 1]))
acc = pa
acc[0]/acc[2]*f+uv_princ[0]

3.6542784242140653

In [129]:
markers = np.array([(3122.4351233671982, 1054.9152394775035),
(2492.4796257168728, 1150.292333232719),
(1874.6113537117903, 1237.0851528384283)])
markers = markers/normf
markers

array([[3.12243512, 1.05491524],
       [2.49247963, 1.15029233],
       [1.87461135, 1.23708515]])

In [130]:
q_i = [qnorm(*uvm, *uv_princ, 1, f) for uvm in markers]

In [131]:
q1, q2, q3 = q_i

In [132]:
k12 = np.dot(q1, q2)
k23 = np.dot(q2, q3)
k31 = np.dot(q3, q1)

l1 = (17.7+16.3)/2
l2 = (10+8.8)/2
l3 = (3+1.2)/2

d12 = l1-l2
d23 = l2-l3
d31 = l1-l3

d122, d232, d312 = [d**2 for d in [d12, d23, d31]]

In [140]:
def func(x):
    x1, x2, x3 = x
    return [x1**2 - 2*k12*x1*x2 + x2**2 - d122, 
            x2**2 - 2*k23*x2*x3 + x3**2 - d232, 
            x3**2 - 2*k31*x3*x1 + x1**2 - d312]

from scipy import optimize
lambda_i = optimize.fsolve(func, np.array([0, 0, 0]))/100

In [141]:
p_i = [l*q for l, q in zip(lambda_i, q_i)]

In [142]:
np.cross(p_i[0] - p_i[1], p_i[1]-p_i[2])

array([-5.86197405e-06, -2.69540967e-05,  3.93644122e-05])

In [143]:
func(lambda_i)

[-57.754223909346926, -53.28467090560558, -221.98779904605115]

In [144]:
q_i

[array([ 0.43549663, -0.56034511,  0.7045254 ]),
 array([ 0.22998269, -0.58212321,  0.77989777]),
 array([-0.01813762, -0.57439631,  0.81837639])]

In [146]:
p_i

[array([ 0.14185374, -0.18252046,  0.22948412]),
 array([ 0.06699758, -0.16958166,  0.2271965 ]),
 array([-0.00497861, -0.15766651,  0.22463681])]

In [145]:
np.dot(table_plane, np.append(p_i[1], 1))

0.006800481666791078

In [148]:
pd = pb + pc - pa
pa, pb, pd, pc

(array([0.11614623, 0.00391899, 0.13034754]),
 array([-0.08056796, -0.04194788,  0.15411815]),
 array([-2.78025045e-04, -3.05224920e-01,  2.89319260e-01]),
 array([ 0.19643617, -0.25935805,  0.26554865]))